In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import pymongo
from pymongo import MongoClient
import numpy as np

file_name = "../geotagged_tweets_20160812-0912.jsons"

In [2]:
user_cols = ['user.name', 'user.screen_name', 'user.followers_count']

place_cols = ['place.full_name', 'place.country_code'] # 'place.bounding_box.coordinates'

entities_cols = ['entities.hashtags', 'entities.user_mentions']

other_cols = ['text', 'lang', 'timestamp_ms']

useful_cols = other_cols + user_cols + place_cols + entities_cols

In [4]:
client = MongoClient()
db = client.twitter
collection = db.inventory
data = pd.DataFrame(list(collection.find()))

In [5]:
data['user'] = data['user'].apply(lambda x: {} if pd.isna(x) else x)
user = json_normalize(data['user'], errors = 'ignore')
user.columns = ['user.' + str(col) for col in user.columns]
user = user[user_cols]

print("User fields flattened")

data['place'] = data['place'].apply(lambda x: {} if pd.isna(x) else x)
place = json_normalize(data['place'], errors = 'ignore')
place.columns = ['place.' + str(col) for col in place.columns]
place = place[place_cols]

print("Place fields flattened")

data['entities'] = data['entities'].apply(lambda x: {} if pd.isna(x) else x)
entities = json_normalize(data['entities'], errors = 'ignore')
entities.columns = ['entities.' + str(col) for col in entities.columns]
entities = entities[entities_cols]

print("Entities fields flattened")

User fields flattened
Place fields flattened
Entities fields flattened


In [6]:
data = data.drop(columns=['user', 'place', 'entities'])
data = pd.concat([data, user, place, entities], axis=1, sort=False)[useful_cols]
data.head()

,text,lang,timestamp_ms,user.name,user.screen_name,user.followers_count,place.full_name,place.country_code,entities.hashtags,entities.user_mentions
0,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,und,1470996240225,GIL DUPUY,DUPUY77,186,"Frontenac, MO",US,[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze..."
1,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,en,1470996242194,Red Octopus,redoctapus,531,"Baton Rouge, LA",US,"[{'text': 'NOJUSTICE', 'indices': [62, 72]}, {...","[{'screen_name': 'BarackObama', 'name': 'Barac..."
2,@HillaryClinton he will do in one year all the...,en,1470996261125,JanJorgenson,jjorgi30,180,"Melbourne, Victoria",AU,[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi..."
3,@theblaze @realDonaldTrump https://t.co/n050DB...,und,1470996250817,GIL DUPUY,DUPUY77,186,"Frontenac, MO",US,[],"[{'screen_name': 'theblaze', 'name': 'TheBlaze..."
4,#CNN #newday clear #Trump deliberately throwin...,en,1470996270035,Beverly Spence,bspence5,2652,"Baltimore, MD",US,"[{'text': 'CNN', 'indices': [0, 4]}, {'text': ...",[]


In [11]:
data.columns

Index(['index', 'text', 'timestamp_ms', 'user.name', 'user.screen_name',
       'user.followers_count', 'place.full_name', 'entities.hashtags',
       'entities.user_mentions'],
      dtype='object')

In [10]:
#Other languages won't work with english libraries, 'und' language seems to be only hashtags, ats, and emojis
data = data[data["lang"] == "en"]
data = data[data["place.country_code"] == "US"]

data = data.drop(columns=['lang', 'place.country_code']).reset_index()
data

KeyError: 'lang'

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(data.groupby("place.full_name").count()["timestamp_ms"].sort_values(ascending=False))

place.full_name
Florida, USA                                                                15590
Manhattan, NY                                                               11464
Pennsylvania, USA                                                           11321
Los Angeles, CA                                                             10887
Georgia, USA                                                                 9228
Kentucky, USA                                                                8833
Chicago, IL                                                                  7633
North Carolina, USA                                                          6754
New York, USA                                                                6658
Texas, USA                                                                   6183
Houston, TX                                                                  5481
Virginia, USA                                                                5302


In [13]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [14]:
# I extract the two letter codes of all the place names in the two different formats
twoletterstates = data["place.full_name"].str.extract(', ([A-Z]{2})$')
fullnamestates = data["place.full_name"].str.extract('(.*), USA')
twoletterstates = pd.DataFrame(twoletterstates[0].combine_first(fullnamestates[0].map(us_state_abbrev)))

In [15]:
twoletterstates[0].value_counts()

CA    61969
FL    50270
NY    44469
TX    33777
PA    23035
NJ    18854
IL    18585
GA    17289
AZ    15976
NC    15609
OH    15029
VA    14503
MA    12028
WA    11721
KY    11599
MI    10557
MD    10302
CO     8623
TN     8431
NV     7237
MO     7090
OR     7021
AL     6828
WI     6707
IN     6561
SC     5834
LA     5019
MS     4651
CT     4552
OK     4146
MN     4066
ME     4051
DC     4016
IA     3883
KS     3144
AR     2816
NH     2654
NM     2560
HI     2503
UT     2305
RI     1889
NE     1842
ID     1802
WV     1475
DE     1450
AK     1255
VT      805
WY      596
MT      582
SD      437
ND      371
Name: 0, dtype: int64

In [66]:
# Add the states and remove useless stuff
df = pd.concat([data, twoletterstates], axis = 1, sort=False)
df = df.drop(columns=['index', 'place.full_name'])
df["timestamp_ms"] = pd.to_datetime(df['timestamp_ms'], unit='ms')
df.rename(columns={'timestamp_ms':'timestamp'}, inplace=True)
df = df.dropna(subset=["state"])

In [67]:
# Take hashtags and mentions outside of the dicts
hashtags = df["entities.hashtags"]
hashtags = hashtags.apply(lambda h : [ x["text"] for x in h ] )
hashtags = hashtags.rename("hashtags")

mentions = df["entities.user_mentions"]
mentions = mentions.apply(lambda h : [ x["screen_name"] for x in h ] )
mentions = mentions.rename("mentions")

In [68]:
# Add hashtags and mentions 
df = pd.concat([df, hashtags, mentions], axis=1, sort=False)
df = df.drop(columns=['entities.hashtags', 'entities.user_mentions'])
df.head()

,text,timestamp,user.name,user.screen_name,user.followers_count,state,hashtags,mentions
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,2016-08-12 10:04:02.194,Red Octopus,redoctapus,531,LA,"[NOJUSTICE, TrumpPence]","[BarackObama, FBI, LorettaLynch, realDonaldTrump]"
1,#CNN #newday clear #Trump deliberately throwin...,2016-08-12 10:04:30.035,Beverly Spence,bspence5,2652,MD,"[CNN, newday, Trump, ISIS]",[]
2,"""Kid, you know, suing someone? Thats the most ...",2016-08-12 10:04:48.229,Rafael Alejandro,GinebraFilms,159,NJ,[],"[funnyordie, realDonaldTrump]"
3,@mike4193496 @realDonaldTrump I TOTALLY CONCUR...,2016-08-12 10:04:53.571,Kim Wasson,kimseacret3,244,MD,[],"[mike4193496, realDonaldTrump]"
4,@HillaryClinton you ARE the co-founder of ISIS...,2016-08-12 10:04:48.695,tom b,VNDISABLEDVET,68,TX,[],[HillaryClinton]


In [94]:
text = df["text"]
text = text.str.replace(r'http?\S+', ' ', regex=True)
#text = text.str.replace(r'[#@]\S+', ' ', regex=True)
text = text.str.replace(r'@\S+', ' ', regex=True)
text = text.str.replace(r'[^\w\s]', ' ', regex=True)
text = text.str.replace(r'\n', ' ', regex=True)
text = text.str.replace(r' +', ' ', regex=True)
text = text.str.lower()

In [98]:
df = df.drop(columns = ["text"])
df = pd.concat([text, df], axis=1, sort=False)
df

,text,timestamp,user.name,user.screen_name,user.followers_count,state,hashtags,mentions
0,all in collusion together nojustice trumppence,2016-08-12 10:04:02.194,Red Octopus,redoctapus,531,LA,"[NOJUSTICE, TrumpPence]","[BarackObama, FBI, LorettaLynch, realDonaldTrump]"
1,cnn newday clear trump deliberately throwing ...,2016-08-12 10:04:30.035,Beverly Spence,bspence5,2652,MD,"[CNN, newday, Trump, ISIS]",[]
2,kid you know suing someone thats the most bea...,2016-08-12 10:04:48.229,Rafael Alejandro,GinebraFilms,159,NJ,[],"[funnyordie, realDonaldTrump]"
3,i totally concur this election is just cra cr...,2016-08-12 10:04:53.571,Kim Wasson,kimseacret3,244,MD,[],"[mike4193496, realDonaldTrump]"
4,you are the co founder of isis you crooked ev...,2016-08-12 10:04:48.695,tom b,VNDISABLEDVET,68,TX,[],[HillaryClinton]
...,...,...,...,...,...,...,...,...
517719,kudlow is on tomorrow what will he say this t...,2016-09-12 13:20:28.065,Linda Mannes Wildes,LindaWildes,2097,FL,[SwordSwallower],"[CNBC, SquawkAlley, realDonaldTrump, Morning_Joe]"
517720,if msm were honest watch any utube video of r...,2016-09-12 13:20:33.874,Karen B,kabpac7,756,FL,[],"[CarolCNN, realDonaldTrump]"
517721,it s interesting that hillary clinton s crowds...,2016-09-12 13:20:38.961,Robert Chaffin,chaffin_robert,2827,AK,[],[]
517722,trump u taxes weird medical report with a whac...,2016-09-12 13:20:32.363,Sheryl Berghoff,Saberghoff,1997,CA,[],[]


In [99]:
df.to_csv("clean.csv")

,text,timestamp,user.name,user.screen_name,user.followers_count,state,hashtags,mentions
0,all in collusion together nojustice trumppence,2016-08-12 10:04:02.194,Red Octopus,redoctapus,531,LA,"[NOJUSTICE, TrumpPence]","[BarackObama, FBI, LorettaLynch, realDonaldTrump]"
1,cnn newday clear trump deliberately throwing ...,2016-08-12 10:04:30.035,Beverly Spence,bspence5,2652,MD,"[CNN, newday, Trump, ISIS]",[]
2,kid you know suing someone thats the most bea...,2016-08-12 10:04:48.229,Rafael Alejandro,GinebraFilms,159,NJ,[],"[funnyordie, realDonaldTrump]"
3,i totally concur this election is just cra cr...,2016-08-12 10:04:53.571,Kim Wasson,kimseacret3,244,MD,[],"[mike4193496, realDonaldTrump]"
4,you are the co founder of isis you crooked ev...,2016-08-12 10:04:48.695,tom b,VNDISABLEDVET,68,TX,[],[HillaryClinton]
...,...,...,...,...,...,...,...,...
517719,kudlow is on tomorrow what will he say this t...,2016-09-12 13:20:28.065,Linda Mannes Wildes,LindaWildes,2097,FL,[SwordSwallower],"[CNBC, SquawkAlley, realDonaldTrump, Morning_Joe]"
517720,if msm were honest watch any utube video of r...,2016-09-12 13:20:33.874,Karen B,kabpac7,756,FL,[],"[CarolCNN, realDonaldTrump]"
517721,it s interesting that hillary clinton s crowds...,2016-09-12 13:20:38.961,Robert Chaffin,chaffin_robert,2827,AK,[],[]
517722,trump u taxes weird medical report with a whac...,2016-09-12 13:20:32.363,Sheryl Berghoff,Saberghoff,1997,CA,[],[]
